# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

from sqlalchemy import create_engine

import pandas as pd
import numpy as np
import re
import nltk
import pickle

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier	
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\G30861\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\G30861\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\G30861\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

##### Load the data from the database

In [10]:
# load data from database
engine = create_engine('sqlite:///test.db')

df = pd.read_sql('categorized_messages', con=engine.connect())
X = df['message'].values
Y = df.iloc[:,4:]

##### Check the dataset for category columns that only has a single class (0 or 1)

In [ ]:
# check for columns that only contain one class
one_class_columns = df.iloc[:,4:].columns[df.iloc[:,4:].nunique() == 1]

In [ ]:
one_class_columns

In [ ]:
# drop columns with only one class from Y Dataset
#try:
#    df.drop(df[one_class_columns], axis=1, inplace=True)
#except:
#    pass

##### Check the dataset for category columns with more than 2 classes (others than 0 or 1)

In [11]:
# check for category columns with m,ore thaqn 2 classes (we expect just two: 0 and 1)
multiple_class_columns = df.iloc[:,4:].columns[df.iloc[:,4:].nunique() > 2]
multiple_class_columns

Index([], dtype='object')

In [ ]:
df['related'].value_counts()

In [ ]:
# check the percentage of values of '2' in column 'related'
df[df['related'] == 2].shape[0] / df.shape[0]

Just 0.7% of the values in the Category Column 'related' are 2. It might be a mistake when typing, nevertheless this percentage has less influence on the model results that we can assume a mistake and replace 2 by 1

In [ ]:
# replace the 2 by 1
#df['related'].replace(2, 1, inplace=True)

In [ ]:
# check the resulting dataframe
#df['related'].value_counts()

##### Split the DataFrame into an X and Y Datasets

In [12]:
# extract the messages column as X and the remaining categories as Y
X = df['message']
Y = df.iloc[:,4:]

In [ ]:
# check X
X

In [ ]:
# check Y
Y

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):

    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()

    #transform to lower case and remoce special characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline_RF = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('mtc', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
#Split data into Train- and Testset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [15]:
#fit pipeline
pipeline_RF.fit(X_train, Y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('mtc',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
# make a prediction on the Testset data
Y_Predict_RF = pipeline_RF.predict(X_test)

In [17]:
# create the classification report and store it into a dataframe
df_class_report_RF = pd.DataFrame(classification_report(Y_test, Y_Predict_RF, output_dict=True,target_names=Y.columns)).transpose()
df_class_report_RF

c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Rec

,precision,recall,f1-score,support
related,0.813734,0.971909,0.885816,3987.0
request,0.905437,0.414054,0.568249,925.0
offer,0.000000,0.000000,0.000000,28.0
aid_related,0.790006,0.601722,0.683128,2207.0
medical_help,0.742857,0.060748,0.112311,428.0
medical_products,0.833333,0.057471,0.107527,261.0
search_and_rescue,1.000000,0.006757,0.013423,148.0
security,0.000000,0.000000,0.000000,103.0
military,1.000000,0.041667,0.080000,192.0
child_alone,0.000000,0.000000,0.000000,0.0


### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
pipeline_RF.get_params()

{'memory': None,
 'steps': [('vectorizer', CountVectorizer()),
  ('tfidf', TfidfTransformer()),
  ('mtc', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vectorizer': CountVectorizer(),
 'tfidf': TfidfTransformer(),
 'mtc': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vectorizer__analyzer': 'word',
 'vectorizer__binary': False,
 'vectorizer__decode_error': 'strict',
 'vectorizer__dtype': numpy.int64,
 'vectorizer__encoding': 'utf-8',
 'vectorizer__input': 'content',
 'vectorizer__lowercase': True,
 'vectorizer__max_df': 1.0,
 'vectorizer__max_features': None,
 'vectorizer__min_df': 1,
 'vectorizer__ngram_range': (1, 1),
 'vectorizer__preprocessor': None,
 'vectorizer__stop_words': None,
 'vectorizer__strip_accents': None,
 'vectorizer__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vectorizer__tokenizer': None,
 'vectorizer__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': Tr

In [21]:
parameters_RF = {
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'mtc__estimator__n_estimators': [50, 100, 200]
}

cv_RF = GridSearchCV(pipeline_RF, param_grid=parameters_RF)

In [22]:
cv_RF.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('vectorizer', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('mtc',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'mtc__estimator__n_estimators': [50, 100, 200],
                         'vectorizer__ngram_range': [(1, 1), (1, 2)]})

In [23]:
cv_RF.best_params_

{'mtc__estimator__n_estimators': 200, 'vectorizer__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
# make a prediction on the Testset data
Y_Predict_cv_RF = cv_RF.predict(X_test)

In [25]:
# create the classification report and store it into a dataframe
df_class_report_cv_RF = pd.DataFrame(classification_report(Y_test, Y_Predict_cv_RF, output_dict=True,target_names=Y.columns)).transpose()
df_class_report_cv_RF

c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Rec

,precision,recall,f1-score,support
related,0.811821,0.974918,0.885926,3987.0
request,0.880694,0.438919,0.585859,925.0
offer,0.000000,0.000000,0.000000,28.0
aid_related,0.809955,0.567739,0.667555,2207.0
medical_help,0.774194,0.056075,0.104575,428.0
medical_products,0.823529,0.053640,0.100719,261.0
search_and_rescue,0.833333,0.033784,0.064935,148.0
security,0.000000,0.000000,0.000000,103.0
military,0.666667,0.010417,0.020513,192.0
child_alone,0.000000,0.000000,0.000000,0.0


The increase of the number of neighbors as a result of the GridSearch improves the model a little bit. Lets try another estimator, maybe KNN:

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
pipeline_Ada = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('mtc', MultiOutputClassifier(AdaBoostClassifier()))
])

In [27]:
#fit pipeline with same Train-Test-Split
pipeline_Ada.fit(X_train, Y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('mtc', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [28]:
# make a prediction on the Testset data
Y_Predict_Ada = pipeline_Ada.predict(X_test)

In [29]:
df_class_report_Ada = pd.DataFrame(classification_report(Y_test, Y_Predict_Ada, output_dict=True,target_names=Y.columns)).transpose()
df_class_report_Ada

c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Rec

,precision,recall,f1-score,support
related,0.807401,0.963130,0.878417,3987.0
request,0.762053,0.529730,0.625000,925.0
offer,0.111111,0.035714,0.054054,28.0
aid_related,0.774076,0.579067,0.662519,2207.0
medical_help,0.596774,0.259346,0.361564,428.0
medical_products,0.620690,0.344828,0.443350,261.0
search_and_rescue,0.463415,0.128378,0.201058,148.0
security,0.210526,0.038835,0.065574,103.0
military,0.598131,0.333333,0.428094,192.0
child_alone,0.000000,0.000000,0.000000,0.0


In [31]:
pipeline_Ada.get_params()

{'memory': None,
 'steps': [('vectorizer', CountVectorizer()),
  ('tfidf', TfidfTransformer()),
  ('mtc', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vectorizer': CountVectorizer(),
 'tfidf': TfidfTransformer(),
 'mtc': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vectorizer__analyzer': 'word',
 'vectorizer__binary': False,
 'vectorizer__decode_error': 'strict',
 'vectorizer__dtype': numpy.int64,
 'vectorizer__encoding': 'utf-8',
 'vectorizer__input': 'content',
 'vectorizer__lowercase': True,
 'vectorizer__max_df': 1.0,
 'vectorizer__max_features': None,
 'vectorizer__min_df': 1,
 'vectorizer__ngram_range': (1, 1),
 'vectorizer__preprocessor': None,
 'vectorizer__stop_words': None,
 'vectorizer__strip_accents': None,
 'vectorizer__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vectorizer__tokenizer': None,
 'vectorizer__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'mt

In [32]:
parameters_Ada = {
    'mtc__estimator__learning_rate': [1.0, 2.0],
    'mtc__estimator__n_estimators': [50, 100],
}

cv_Ada = GridSearchCV(pipeline_Ada, param_grid=parameters_Ada)

In [33]:
cv_Ada.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('vectorizer', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('mtc',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'mtc__estimator__learning_rate': [1.0, 2.0],
                         'mtc__estimator__n_estimators': [50, 100]})

In [34]:
cv_Ada.best_params_

{'mtc__estimator__learning_rate': 1.0, 'mtc__estimator__n_estimators': 100}

In [35]:
# make a prediction on the Testset data
Y_Predict_cv_Ada = cv_Ada.predict(X_test)

In [36]:
# create the classification report and store it into a dataframe
df_class_report_cv_Ada = pd.DataFrame(classification_report(Y_test, Y_Predict_cv_Ada, output_dict=True,target_names=Y.columns)).transpose()
df_class_report_cv_Ada

c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Rec

,precision,recall,f1-score,support
related,0.816962,0.947078,0.877222,3987.0
request,0.773556,0.550270,0.643083,925.0
offer,0.066667,0.035714,0.046512,28.0
aid_related,0.773585,0.613049,0.684024,2207.0
medical_help,0.597087,0.287383,0.388013,428.0
medical_products,0.595890,0.333333,0.427518,261.0
search_and_rescue,0.463415,0.128378,0.201058,148.0
security,0.250000,0.038835,0.067227,103.0
military,0.586538,0.317708,0.412162,192.0
child_alone,0.000000,0.000000,0.000000,0.0


The change of the estimator to a Logistic Regression further increases the model!

### 9. Export your model as a pickle file

Due to the very long runtime of the random forest GridSearch compared to the boosted model, I will save the AdaBoost model.

In [37]:
with open('disaster_classifier_Ada.pkl', 'wb') as file:
    pickle.dump(cv_Ada, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.